In [ ]:
import pymysql
from tkinter import *
from tkinter import messagebox, ttk
import tkinter as tk
from PIL import Image, ImageTk
import tkcalendar
from tkcalendar import DateEntry
from tkcalendar import Calendar
from datetime import date
import random
import string
# ---------- DATABASE CONFIG ----------
DB_HOST = 'localhost'
DB_NAME = 'data'
DB_USER = 'root'
DB_PASS = 'Dharshini'  # your MySQL password


# ---------- DATABASE HELPER ----------
def get_connection():
    try:
        return pymysql.connect(
            host=DB_HOST,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASS
        )
    except Exception as e:
        messagebox.showerror("Database Error", f"Could not connect to DB:\n{e}")
        return None

def generate_student_id():
           random_number = random.randint(10000, 99999)
           ticket_number = f"BTS{random_number}" 
           return ticket_number
# ---------- MAIN APP ----------
class CRUDApp(Tk):
    def __init__(self):
        super().__init__()
        self.title("CRUD Operation in Python")
        self.geometry("1100x700")
        self.configure(bg="white")

        self.logged_in_user = None
        
       


        # ---------- MENU BAR ----------
        menubar = Menu(self)
        

        # Student Menu
        booking_menu = Menu(menubar, tearoff=0,font=("Times New Roman", 12, "bold"))
        booking_menu.add_command(label="Admin", command=self.view_students)
        #booking_menu.add_command(label="Book Ticket", command=self.add_student_window)
        #booking_menu.add_command(label="Update Ticket", command=self.update_student_window)
        #booking_menu.add_command(label="Cancel Ticket", command=self.delete_student_window)
        booking_menu.add_separator()
        booking_menu.add_command(label="Exit", command=self.quit)
        # -------- USER MENU --------
        user_menu = Menu(menubar, tearoff=0, font=("Times New Roman", 12, "bold"))
        user_menu.add_command(label="Register", command=self.register_window)
        user_menu.add_command(label="Login", command=self.login_window)
        user_menu.add_command(label="Book Ticket", command=self.add_student_window)
        user_menu.add_command(label="Update Ticket", command=self.update_student_window)
        user_menu.add_command(label="Cancel Ticket", command=self.delete_student_window)
        user_menu.add_command(label="My Bookings", command=self.my_bookings)
        user_menu.add_command(label="Profile", command=self.view_profile)
        user_menu.add_command(label="Booking History", command=self.booking_history)

        user_menu.add_separator()
        user_menu.add_command(label="Logout", command=self.logout)

        menubar.add_cascade(label="User", menu=user_menu, font=("Times New Roman", 14, "bold"))

        menubar.add_cascade(label="Admin", menu=booking_menu,font=("Times New Roman", 14, "bold"))
        

        # Attach menubar to window
        self.config(menu=menubar)


        header_frame = tk.Frame(self, bg="#f8f9fa")
        header_frame.pack(pady=(20, 0), fill="x")

        try:
            logo_img = Image.open(r"C:\Users\91637\OneDrive\Desktop\jorim internship\logopic.jpg")
            logo_img = logo_img.resize((200, 90))
            logo_photo = ImageTk.PhotoImage(logo_img)
            logo_label = tk.Label(header_frame, image=logo_photo, bg="#f8f9fa")
            logo_label.image = logo_photo 
            logo_label.pack(side=tk.LEFT, padx=(40, 50))
        except Exception:
            # If there's any issue loading the logo, just continue without it
            pass


        Label(header_frame, text="    Kaaliyapaa Travels Bus Ticket Booking System   ",
              fg="black", bg="#f8f9fa", font=("Times New Roman", 30, "bold")).pack(side=tk.LEFT, pady=10, fill="x")
        
        
        #buttons
        btn_frame = Frame(self, bg="white")
        btn_frame.pack(pady=5)

        

        #------Main Form Frame------

        self.form_frame = Frame(self, bg="#f5f6fa")
        self.form_frame.pack(fill="x",pady=10)


        self.grid_rowconfigure(2, weight=1)
        self.grid_columnconfigure(0, weight=1)
        self.form_frame.grid_rowconfigure(0, weight=1)
        self.form_frame.grid_columnconfigure(0, weight=1)



    def clear_form_frame(self):
        for widget in self.form_frame.winfo_children():
            widget.destroy()  
        
    

    # ---------- VIEW ----------
    def view_students(self):
        # Create Treeview inside form_frame
        self.clear_form_frame()


        #---customize treeview style---

        style = ttk.Style()
        style.theme_use("clam")  # clam theme allows color changes

         # Customize treeview headings
        style.configure("Custom.Treeview.Heading",font=("Times New Roman", 14, "bold"), foreground="white",background="#5298c7" )

         # Customize rows
        style.configure("Custom.Treeview",font=("Times New Roman", 12),rowheight=30,background="#f5f6fa",fieldbackground="#f5f6fa")
    

        style.map("Custom.Treeview.Heading", background=[("active", "#155B8E")] )


        tk.Label(self.form_frame, text="Enter Admin Password",
                font=("Times New Roman", 12), bg="#f5f6fa").grid(row=0, column=0, pady=2,padx=5, sticky="n")
        pwd_entry = ttk.Entry(self.form_frame, show="*", width=30)
        pwd_entry.grid(row=0, column=1, padx=5,pady=2,ipady=5)

        self.form_frame.grid_rowconfigure(2, weight=1)
        self.form_frame.grid_columnconfigure(0, weight=1)
        self.form_frame.grid_columnconfigure(1, weight=1)
        self.form_frame.grid_columnconfigure(2, weight=1)
                       
        self.listTree = ttk.Treeview(self.form_frame, columns=("Ticket Number","Name", "Gender", "Age", "Boarding", "Dropping", "Date","Contact Number"), show="headings",height=15,style="Custom.Treeview")
        #self.listTree.heading("Stud_ID", text="Ticket Number")
        #self.listTree.heading("name", text="Name")
        #self.listTree.heading("gender", text="Gender")
        #self.listTree.heading("age", text="Age")
        #self.listTree.heading("boarding", text="Boarding Point")
        #self.listTree.heading("dropping", text="Dropping Point")
        #self.listTree.heading("date", text="Date")
        #self.listTree.heading("Contact Number", text="Contact Number")


        for col in self.listTree["columns"]:
           self.listTree.heading(col, text=col)
           self.listTree.column(col, anchor="w", width=140)

        self.listTree.grid(row=2, column=0,columnspan=2,padx=70, pady=20, sticky="nsew")
        self.listTree.grid_remove()

        #---function to check password and show table---
        def check_password():
            for widget in self.form_frame.grid_slaves(row=3, column=0):
               widget.destroy()
            if pwd_entry.get() != DB_PASS:
                error_label = tk.Label(self.form_frame, text="Access Denied: Only admin can view records.", fg="red", bg="#f8f9fa", font=("Segoe UI", 12, "bold"))
                error_label.grid(row=3, column=0, columnspan=2, pady=10)
                return
            
            self.listTree.grid()

            conn = get_connection()
            if not conn:
              return

            cursor = conn.cursor()
            cursor.execute("SELECT * FROM cust")
            rows = cursor.fetchall()
            self.listTree.delete(*self.listTree.get_children())
            for row in rows:
               self.listTree.insert("", "end", text=row[0], values=row)
            conn.close()
            

        ttk.Button(self.form_frame, text="Submit", command=check_password).grid(row=1, column=1, pady=10, sticky="n")



    # ---------- ADD ----------
    def add_student_window(self):
        
        
        self.clear_form_frame()
        self.form_frame.configure(bg="#f5f6fa")
        
        container = Frame(self.form_frame, bg="#f5f6fa")
        container.place(relx=0.5, rely=0.4, anchor="center")

        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)
        container.grid_columnconfigure(1, weight=1)

        labels = ["Name", "Gender", "Age", "Boarding Point", "Dropping Point", "Date","Contact Number"]
                  
        entries = {}
        error_labels = {}   

        self.form_frame.update_idletasks()

        boarding_points = ["Nanganallur", "Madipakam", "Koyembedu", "Kelambakam", "Tnagar","Tambaram"]
        dropping_points = ["Nanganallur", "Madipakam", "Koyembedu", "Kelambakam", "Tnagar","Tambaram"]

        for i, lbl in enumerate(labels):
            Label(self.form_frame, text=lbl + "  :",bg="#f5f6fa",font=("Times New Roman",11,"bold")).grid(row=i, column=0, pady=5,padx=(15,0),sticky="e") 

            if lbl == "Date":
               #cal = Calendar(self.form_frame,selectmode="day",date_pattern="dd/mm/yyyy",background="white",disabledbackground="white",bordercolor="#d1d9e6",
               #headersbackground="#5298c7",headersforeground="white",normalbackground="white",weekendbackground="#f5f6fa",
               #othermonthbackground="#e6e6e6",othermonthwebackground="#e6e6e6",selectbackground="#155B8E",
               #selectforeground="white",font=("Times New Roman", 12),headersfont=("Times New Roman", 13, "bold"),mindate=date.today())
               #cal.grid(row=i, column=1, padx=10, pady=8, ipady=5)



                ent = DateEntry(self.form_frame, width=20, 
                            foreground="#94834a", borderwidth=2, date_pattern="dd/mm/yyyy",background="white",disabledbackground="white",bordercolor="#d1d9e6",
                            headersbackground="#5298c7",headersforeground="white",normalbackground="white",weekendbackground="#f5f6fa",
                            othermonthbackground="#e6e6e6",othermonthwebackground="#e6e6e6",selectbackground="#155B8E",
                            selectforeground="white",font=("Times New Roman", 12),headersfont=("Times New Roman", 13, "bold"),mindate=date.today())

            elif lbl == "Gender":
                ent = ttk.Combobox(self.form_frame, values=["Male", "Female", "Others"], state="readonly", width=27)
                ent.set("Select Gender")

            elif lbl == "Boarding Point":
                ent = ttk.Combobox(self.form_frame, values=boarding_points,state="readonly", width=27)
                ent.set("Select Boarding Point")

            elif lbl == "Dropping Point":
               ent = ttk.Combobox(self.form_frame, values=dropping_points,state="readonly", width=27)
               ent.set("Select Dropping Point")

            else:
               ent = Entry(self.form_frame, width=30,bg="#f7f7f7",fg="black",insertbackground="black",highlightthickness=2,highlightbackground="#f5f6fa",highlightcolor="#89cff3")
        

            ent.grid(row=i, column=1,padx=(20,5),pady=5,ipady=5,sticky="w")
            entries[lbl] = ent

            error_label = Label(self.form_frame, text="", fg="red",bg="#f5f6fa",font=("Times New Roman",9 ,"bold"))
            error_label.grid(row=i, column=2, columnspan=2,pady=5,padx=(0,5),sticky="w")
            error_labels[lbl] = error_label


            # --- VALIDATION FUNCTIONS ---
        def validate_name(_event=None):
            name = entries["Name"].get().strip()
            if len(name) < 3:
                error_labels["Name"].config(text="⚠ At least 3 characters")
            else:
                error_labels["Name"].config(text="")

        def validate_boarding(_event=None):
            value = entries["Boarding Point"].get()
            if value == "Select Boarding Point":
                error_labels["Boarding Point"].config(text="⚠ Please select a boarding point")
            else:
                error_labels["Boarding Point"].config(text="")

        def validate_dropping(_event=None):
            value = entries["Dropping Point"].get()
            if value == "Select Dropping Point":
                error_labels["Dropping Point"].config(text="⚠ Please select a dropping point")
            elif value == entries["Boarding Point"].get():
                error_labels["Dropping Point"].config(text="⚠ Boarding and Dropping cannot be same")
            else:
                error_labels["Dropping Point"].config(text="")

        def validate_gender(_event=None):
            value = entries["Gender"].get()
            if value == "Select Gender":
                error_labels["Gender"].config(text="⚠ Please select a gender")
            else:
                error_labels["Gender"].config(text="")


        def validate_contact(_event=None):
            contact = entries["Contact Number"].get().strip()
            if not contact.isdigit() or len(contact) != 10:
               error_labels["Contact Number"].config(text="⚠ Must be 10 digits")
            else:
               error_labels["Contact Number"].config(text="")

        def validate_age(_event=None):
            age = entries["Age"].get().strip()
            if not age.isdigit() or int(age) <= 0:
               error_labels["Age"].config(text="⚠ Enter valid age")
            else:
               error_labels["Age"].config(text="")

    # --- Bind validators ---
        entries["Name"].bind("<FocusOut>", validate_name)
        entries["Name"].bind("<KeyRelease>", validate_name)
        entries["Boarding Point"].bind("<FocusOut>", validate_boarding)
        entries["Boarding Point"].bind("<KeyRelease>", validate_boarding)
        entries["Dropping Point"].bind("<FocusOut>", validate_dropping)
        entries["Dropping Point"].bind("<KeyRelease>", validate_dropping)
        entries["Gender"].bind("<FocusOut>", validate_gender)
        entries["Gender"].bind("<KeyRelease>", validate_gender)
        entries["Contact Number"].bind("<FocusOut>", validate_contact)
        entries["Contact Number"].bind("<KeyRelease>", validate_contact)
        entries["Age"].bind("<FocusOut>", validate_age)
        entries["Age"].bind("<KeyRelease>", validate_age)



    
        def save_student():
            data = [entries[lbl].get().strip() for lbl in labels]
            error_label.config(text="")
            if any(val == "" or val =="Select Gender" for val in data):
                error_label.config(text=" ⚠️ All fields are required!")
                return
            if len(entries["Name"].get().strip()) < 3:
                error_label.config(text="⚠️ Name must be at least 3 characters long!")
                return
            if len(entries["Contact Number"].get().strip()) !=10 :
                error_label.config(text="⚠️ Contact Number must be 10 digits!")
                return
            
            student_id = generate_student_id()

            conn = get_connection()
            if not conn:
                return
            cursor = conn.cursor()
            try:
                while True:
                    student_id = generate_student_id()
                    cursor.execute("SELECT COUNT(*) FROM cust WHERE Stud_ID = %s", (student_id,))
                    if cursor.fetchone()[0] == 0:
                      break

                print("Data being inserted:", [student_id] + data)
                print("Length of data:", len([student_id] + data))

                cursor.execute("""
                    INSERT INTO cust(Stud_ID,Name, Gender, Age, BoardingPoint, DroppingPoint, Date,contact,user_id)
                    VALUES (%s,%s, %s, %s, %s, %s, %s,%s,%s)
                """,[student_id]+ data + [self.logged_in_user["user_id"]])
                conn.commit()
                status_label.config(text="✅ Booked successfully!", fg="green")

                
            except Exception as e:
                status_label.config(text=f"⚠️ Could not add: {e}", fg="red")

            finally:
                conn.close()

        def clear_form():
            for label, widget in entries.items():
                if label == "Date":  
                    widget.set_date(date.today()) 
                elif isinstance(widget, ttk.Combobox):
                    widget.set(f"Select {label}")
                else:
                    widget.delete(0, tk.END) 

        Button(self.form_frame, text="Clear", bg="#f5f6fa",width=15,font=("Times New Roman", 13, "bold"), command=clear_form).grid(
            row=len(labels)+1, column=0,pady=10,padx=2,ipady=5,sticky="e")

        Button(self.form_frame, text="Submit", bg="#f5f6fa",width=15,font=("Times New Roman", 13, "bold"), command=save_student).grid(
            row=len(labels)+1, column=1,pady=10,padx=(40,2),ipady=5,sticky="w")
        
        
        #error_label = Label(self.form_frame, text="", fg="red", bg="#f5f6fa",font=("Times New Roman", 9))
        #error_label.grid(row=len(labels)*2, column=0, columnspan=2,pady=10,padx=10)
        #error_labels = error_label
        
            
        status_label = Label(self.form_frame, text="", fg="green", bg="#f5f6fa", font=("Times New Roman", 9, "bold"))
        status_label.grid(row=len(labels)+2, column=0, columnspan=2, pady=10)

    


    # ---------- UPDATE ----------
    def update_student_window(self):
        self.clear_form_frame()
        self.form_frame.configure(bg="#f5f6fa")

        
        self.form_frame.grid_rowconfigure(0, weight=1)
        self.form_frame.grid_columnconfigure(0, weight=1)
        self.form_frame.grid_columnconfigure(1, weight=1)

        Label(self.form_frame, text="Enter ID",bg="#f5f6fa",font=("Times New Roman",12,"bold")).grid(row=0, column=0, pady=5.5,padx=(15,0),sticky="e")
        id_entry = Entry(self.form_frame, width=30)
        id_entry.grid(row=0, column=1,padx=(20,2),pady=5.5,ipady=5,sticky="w")

        labels = ["Name",  "Gender", "Age", "Boarding Point", "Dropping Point", "Date","Contact Number"]
            
        entries = {}
        error_labels = {}

        boarding_points = ["Thanjavur", "Trichy", "Coimbatore", "Chennai", "Pandicherry","Madurai","Salem","Vellore"]
        dropping_points = ["Thanjavur", "Trichy", "Coimbatore", "Chennai", "Pandicherry","Madurai","Salem","Vellore"]

        for i, lbl in enumerate(labels, start=1):
            Label(self.form_frame, text=lbl+ "  :",bg="#f5f6fa",font=("Times New Roman",12,"bold")).grid(row=i, column=0, pady=5.5,padx=(15,0),sticky="e")
            if lbl == "Date":
                ent = DateEntry(self.form_frame, width=20, 
                            foreground="#94834a", borderwidth=2, date_pattern="dd/mm/yyyy",background="white",disabledbackground="white",bordercolor="#d1d9e6",
                            headersbackground="#5298c7",headersforeground="white",normalbackground="white",weekendbackground="#f5f6fa",
                            othermonthbackground="#e6e6e6",othermonthwebackground="#e6e6e6",selectbackground="#155B8E",
                            selectforeground="white",font=("Times New Roman", 12),headersfont=("Times New Roman", 13, "bold"),mindate=date.today())
            elif lbl == "Gender":
                ent = ttk.Combobox(self.form_frame, values=["Male", "Female", "Others"], state="readonly", width=27)
                ent.set("Select Gender")

            elif lbl == "Boarding Point":
                ent = ttk.Combobox(self.form_frame, values=boarding_points,state="readonly", width=27)
                ent.set("Select Boarding Point")

            elif lbl == "Dropping Point":
               ent = ttk.Combobox(self.form_frame, values=dropping_points,state="readonly", width=27)
               ent.set("Select Dropping Point")

            else:
                ent = Entry(self.form_frame, width=30,bg="#f7f7f7",fg="black",insertbackground="black",highlightthickness=2,highlightbackground="#f5f6fa",highlightcolor="#89cff3")
            ent.grid(row=i, column=1,padx=(20,2),pady=5.5,ipady=5,sticky="w")
            entries[lbl] = ent

            error_label = Label(self.form_frame, text="", fg="red", bg="#f5f6fa",font=("Times New Roman", 9,"bold"))
            error_label.grid(row=i, column=2, columnspan=2,pady=10,padx=(0,5),sticky="w")
            error_labels[lbl] = error_label


            # --- VALIDATION FUNCTIONS ---
        def validate_name(_event=None):
            name = entries["Name"].get().strip()
            if len(name) < 3:
                error_labels["Name"].config(text="⚠ At least 3 characters")
            else:
                error_labels["Name"].config(text="")

        def validate_contact(_event=None):
            contact = entries["Contact Number"].get().strip()
            if not contact.isdigit() or len(contact) != 10:
               error_labels["Contact Number"].config(text="⚠ Must be 10 digits")
            else:
               error_labels["Contact Number"].config(text="")

        def validate_boarding(_event=None):
            value = entries["Boarding Point"].get()
            if value == "Select Boarding Point":
                error_labels["Boarding Point"].config(text="⚠ Please select a boarding point")
            else:
                error_labels["Boarding Point"].config(text="")

        def validate_dropping(_event=None):
            value = entries["Dropping Point"].get()
            if value == "Select Dropping Point":
                error_labels["Dropping Point"].config(text="⚠ Please select a dropping point")
            elif value == entries["Boarding Point"].get():
                error_labels["Dropping Point"].config(text="⚠ Boarding and Dropping cannot be same")
            else:
                error_labels["Dropping Point"].config(text="")

        def validate_gender(_event=None):
            value = entries["Gender"].get()
            if value == "Select Gender":
                error_labels["Gender"].config(text="⚠ Please select a gender")
            else:
                error_labels["Gender"].config(text="")

        def validate_age(_event=None):
            age = entries["Age"].get().strip()
            if not age.isdigit() or int(age) <= 0:
               error_labels["Age"].config(text="⚠ Enter valid age")
            else:
               error_labels["Age"].config(text="")

    # --- Bind validators ---
        entries["Name"].bind("<FocusOut>", validate_name)
        entries["Name"].bind("<KeyRelease>", validate_name)
        entries["Boarding Point"].bind("<FocusOut>", validate_boarding)
        entries["Boarding Point"].bind("<KeyRelease>", validate_boarding)
        entries["Dropping Point"].bind("<FocusOut>", validate_dropping)
        entries["Dropping Point"].bind("<KeyRelease>", validate_dropping)
        entries["Gender"].bind("<FocusOut>", validate_gender)
        entries["Gender"].bind("<KeyRelease>", validate_gender)
        entries["Contact Number"].bind("<FocusOut>", validate_contact)
        entries["Contact Number"].bind("<KeyRelease>", validate_contact)
        entries["Age"].bind("<FocusOut>", validate_age)
        entries["Age"].bind("<KeyRelease>", validate_age)

        
        def search_student():
            sid = id_entry.get().strip()
            if sid == "":
                error_label.config(text="Error: Enter  ID")
                return
            conn = get_connection()
            if not conn:
                return
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM cust WHERE Stud_ID=%s", (sid,))
            row = cursor.fetchone()
            conn.close()
            if row:
                for lbl, val in zip(labels, row[1:]):
                    widget = entries[lbl]
                    widget_type = widget.winfo_class()
                    if widget_type == "Entry":
                        widget.delete(0, END)
                        if val is not None:
                           widget.insert(0, str(val))
                    elif widget_type == "TCombobox":
                        widget.set(val if val is not None else "")
                    elif widget_type == "DateEntry":
                        if val is not None:
                           widget.set_date(val)
            else:
                error_label.config(text=" Details not found!")

        def update_student():
            sid = id_entry.get().strip()
            if sid == "":
                error_label.config(text="Error :Enter  ID")
                return
            data = [entries[lbl].get().strip() for lbl in labels] + [sid]

            error_label.config(text="")
            if any(val == "" or val =="Select Gender" for val in data):
                error_label.config(text="  All fields are required!")
                return
            if len(entries["Name"].get().strip()) < 3:
                error_label.config(text=" Name must be at least 3 characters long!")
                return
            if len(entries["Contact Number"].get().strip()) != 10 :
                error_label.config(text="Contact Number must be 10 digits!")
                return
            
            if any(val == "" for val in data):
                error_label.config(text="Error: All fields are required!")
                return
            conn = get_connection()
            if not conn:
                return
            cursor = conn.cursor()
            try:
                cursor.execute("""
                    UPDATE cust
                    SET Name=%s, Gender=%s, Age=%s, BoardingPoint=%s, DroppingPoint=%s, Date=%s,Contact=%s
                    WHERE Stud_ID=%s
                """, data)
                conn.commit()
                status_label.config(text= " Updated successfully!",fg="green")
            except Exception as e:
                error_label.config(text="Could not update :\n{e}")
            finally:
                conn.close()

        Button(self.form_frame, text="Search", bg="#f5f6fa",width=15,font=("Times New Roman", 13, "bold"),
               command=search_student).grid(row=len(labels)+1, column=0, pady=10,padx=2,ipady=5,sticky="e")
        Button(self.form_frame, text="Update", bg="#f5f6fa",width=15,font=("Times New Roman", 13, "bold"),
               command=update_student).grid(row=len(labels)+1, column=1, pady=10,padx=(40,2),ipady=5,sticky="w")
        
        
        error_label = Label(self.form_frame, text="", fg="red", bg="#f5f6fa")
        error_label.grid(row=len(labels)+2, column=0, columnspan=2, pady=5)

        status_label = Label(self.form_frame, text="", fg="green", bg="#f5f6fa", font=("Times New Roman", 10, "bold"))
        status_label.grid(row=len(labels)*3, column=0, columnspan=2, pady=10)

    # ---------- DELETE ----------
    def delete_student_window(self):
        self.clear_form_frame()

        self.form_frame.grid_columnconfigure(0, weight=1)
        self.form_frame.grid_columnconfigure(1, weight=0)
        self.form_frame.grid_columnconfigure(2, weight=1)

        Label(self.form_frame, text="Enter  ID",width=30,bg="#f5f6fa",font=("Times New Roman", 14, "bold")).grid(row=0, column=1, pady=10,padx=8,ipady=5)
        id_entry = Entry(self.form_frame, width=35,bg="#f7f7f7",fg="black",insertbackground="black",highlightthickness=2,highlightbackground="#f5f6fa",highlightcolor="#89cff3")
        id_entry.grid(row=1, column=1, pady=10,padx=8,ipady=5)

        def show_details():
            sid = id_entry.get().strip()
            if sid == "":
               error_label.config(text="Error: Enter ID")
               return
            conn = get_connection()
            if not conn:
                return
            cursor = conn.cursor()
            try:
                cursor.execute("SELECT * FROM cust WHERE Stud_ID=%s", (sid,))
                details = cursor.fetchone()
                if details:
                   details = f"Ticket Number: {details[0]}\n\nName: {details[1]}\n\nGender: {details[2]}\n\nAge: {details[3]}\n\nBoarding: {details[4]}\n\nDropping: {details[5]}\n\nDate: {details[6]}"
                   status_label.config(text=details, fg="black", font=("Times New Roman", 12, "bold"), justify="left")
                else:
                   status_label.config(text="No record found for this ID", fg="red")
            except Exception as e:
                status_label.config(text=f"Error fetching details: {e}", fg="red")
            finally:
                conn.close()
        


        def delete_student():
            sid = id_entry.get().strip()
            if sid == "":
                error_label.config(text="Error: Enter  ID")
                return
            conn = get_connection()
            if not conn:
                return
            cursor = conn.cursor()
            confirm = messagebox.askyesno("Confirm Delete", 
                              f"Do you want to cancel the ticket for ID {sid}?")
 
            if confirm:   # User clicked "Yes"
              cursor.execute("DELETE FROM cust WHERE Stud_ID=%s", (sid,))
              conn.commit()
              error_label.config(text="Ticket Cancelled successfully! Refund will be processed soon.", font=("Times New Roman", 12, "bold"))
             
            else:
              error_label.config(text="Cancellation aborted.", font=("Times New Roman", 12, "bold"))

       # Button(self.form_frame, text="Show Details", bg="#baebff", fg="black", width=15,
        #   font=("Times New Roman", 11, "bold"), command=show_details).grid(row=2, column=1, pady=10, padx=2, ipady=5,sticky="n")
        
        #Button(self.form_frame, text="Delete", bg="red",width=15,font=("Times New Roman", 11, "bold"),
         #      command=delete_student).grid(row=2, column=2, pady=10,padx=2,ipady=5,sticky="w")

        btn_frame = Frame(self.form_frame, bg="#f5f6fa")
        btn_frame.grid(row=2, column=1, pady=10)

        show_button = Button(btn_frame, text="Show Details", bg="#f5f6fa", width=15, font=("Times New Roman", 11, "bold"),command=show_details)
        show_button.pack(side=LEFT, padx=5)

        delete_btn = Button(btn_frame, text="Delete", bg="red", width=15, font=("Times New Roman", 11, "bold"),command=delete_student)
        delete_btn.pack(side=LEFT, padx=5)  # 5 px ≈ 0.5 cm

        
        
        error_label = Label(self.form_frame, text="", fg="red", bg="#f5f6fa")
        error_label.grid(row=3, column=0, columnspan=3, pady=5)

        status_label = Label(self.form_frame, text="", fg="green", bg="#f5f6fa", font=("Times New Roman", 10, "bold"))
        status_label.grid(row=4, column=0, columnspan=4, pady=10)

    # ---------- USER LOGIN ----------

    def login_window(self):
        self.clear_form_frame()
        self.form_frame.configure(bg="#f5f6fa")

        Label(self.form_frame, text="User Login", font=("Times New Roman", 18, "bold"), bg="#f5f6fa").grid(row=0, column=0, columnspan=2, pady=10,padx=(20,2),ipady=5,sticky="n")
        Label(self.form_frame, text="Mobile Number:", font=("Times New Roman", 12,"bold"), bg="#f5f6fa").grid(row=1, column=0,ipady=5,pady=5.5,padx=(15,0),sticky="e")
        mobile_entry = Entry(self.form_frame, width=35)
        mobile_entry.grid(row=1, column=1, pady=5.5,padx=(20,2),ipady=5,sticky="w")

        Label(self.form_frame, text="Password:", font=("Times New Roman", 12,"bold"), bg="#f5f6fa").grid(row=2, column=0,ipady=5,pady=5.5,padx=(15,0),sticky="e")
        password_entry = Entry(self.form_frame, width=35, show="*")
        password_entry.grid(row=2, column=1,pady=5.5,padx=(20,2),ipady=5,sticky="w")

        status_label = Label(self.form_frame, text="", fg="red", bg="#f5f6fa", font=("Times New Roman", 10))
        status_label.grid(row=4, column=0, columnspan=2, pady=5)

        def validate_login():
            mobile = mobile_entry.get().strip()
            password = password_entry.get().strip()

            if not mobile or not password:
               status_label.config(text="⚠ Please enter all fields.")
               return

            conn = get_connection()
            if not conn:
                return
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM users WHERE contact=%s AND password=%s", (mobile, password))
            user = cursor.fetchone()
            conn.close()

            if user:
                self.logged_in_user = {
                    "user_id": user[0],
                    "name": user[1],
                    "contact": user[2],
                    "email": user[4]
                 }
                status_label.config(text="✅ Login successful!", fg="green")
                self.my_bookings()
            else:
                 status_label.config(text="❌ Invalid mobile or password.", fg="red")

        Button(self.form_frame, text="Login", font=("Times New Roman", 12, "bold"),
             command=validate_login).grid(row=3, column=0, columnspan=2, pady=10)

    def my_bookings(self):
        if not self.logged_in_user:
           messagebox.showwarning("Login Required", "Please login to view your bookings.")
           return

        self.clear_form_frame()
        Label(self.form_frame, text=f"My Bookings ({self.logged_in_user['name']})",
           font=("Times New Roman", 16, "bold"), bg="#f5f6fa").pack(pady=10)

        tree = ttk.Treeview(self.form_frame, columns=("Ticket Number", "Name", "Boarding Point", "Dropping Point", "Date"), show="headings", height=10)
        for col in tree["columns"]:
           tree.heading(col, text=col)
           tree.column(col, width=150)
        tree.pack(pady=10)

        conn = get_connection()
        if not conn:
          return
        cursor = conn.cursor()
        cursor.execute("SELECT Stud_ID, Name, BoardingPoint, DroppingPoint, Date FROM cust WHERE user_id=%s AND STR_TO_DATE(Date, '%%d/%%m/%%Y') >= CURDATE() ORDER BY STR_TO_DATE(Date, '%%d/%%m/%%Y') DESC",
                   (self.logged_in_user["user_id"],))
        rows = cursor.fetchall()
        conn.close()

        for row in rows:
           tree.insert("", "end", values=row)

    def booking_history(self):
        if not self.logged_in_user:
           messagebox.showwarning("Login Required", "Please login to view booking history.")
           return

        self.clear_form_frame()
        Label(self.form_frame, text="Booking History", font=("Times New Roman", 16, "bold"), bg="#f5f6fa").pack(pady=10)

        tree = ttk.Treeview(self.form_frame, columns=("Ticket", "Name", "Boarding", "Dropping", "Date"), show="headings", height=10)
        for col in tree["columns"]:
            tree.heading(col, text=col)
            tree.column(col, width=150)
        tree.pack(pady=10)

        conn = get_connection()
        if not conn:
            return
        cursor = conn.cursor()
        cursor.execute("SELECT Stud_ID, Name, BoardingPoint, DroppingPoint, Date FROM cust WHERE user_id=%s AND STR_TO_DATE(Date, '%%d/%%m/%%Y') < CURDATE() ORDER BY STR_TO_DATE(Date, '%%d/%%m/%%Y') DESC",
                   (self.logged_in_user["user_id"],))
        rows = cursor.fetchall()
        conn.close()

        for row in rows:
            tree.insert("", "end", values=row)

    def view_profile(self):
        if not self.logged_in_user:
            messagebox.showwarning("Login Required", "Please login to view your profile.")
            return

        self.clear_form_frame()
        user = self.logged_in_user

        Label(self.form_frame, text="My Profile", font=("Times New Roman", 16, "bold"), bg="#f5f6fa").grid(row=0, column=0, columnspan=2, pady=15)

        Label(self.form_frame, text="Contact   :",font=("Times New Roman", 14,"bold"), bg="#f5f6fa").grid(row=1, column=0, ipady=5,pady=5.5,padx=(15,0),sticky="e")
        Label(self.form_frame, text=user["contact"],font=("Times New Roman", 14,"bold"), bg="#f5f6fa").grid(row=1, column=1,pady=5.5,padx=(20,2),ipady=5,sticky="w")

        Label(self.form_frame, text="Name   :",font=("Times New Roman", 14,"bold"), bg="#f5f6fa").grid(row=2, column=0,ipady=5,pady=5.5,padx=(15,0),sticky="e")
        name_entry = Entry(self.form_frame, width=30)
        name_entry.insert(0, user["name"])
        name_entry.grid(row=2, column=1, pady=5.5,padx=(20,2),ipady=5,sticky="w")

        Label(self.form_frame, text="Email   :",font=("Times New Roman", 14,"bold"), bg="#f5f6fa").grid(row=3, column=0, ipady=5,pady=5.5,padx=(15,0),sticky="e")
        email_entry = Entry(self.form_frame, width=30)
        email_entry.insert(0, user.get("email", ""))
        email_entry.grid(row=3, column=1,pady=5.5,padx=(20,2),ipady=5,sticky="w")

        def update_profile():
            conn = get_connection()
            if not conn:
               return
            cursor = conn.cursor()
            cursor.execute("UPDATE users SET name=%s, email=%s WHERE user_id=%s",
                       (name_entry.get().strip(), email_entry.get().strip(), user["user_id"]))
            conn.commit()
            conn.close()
            self.logged_in_user["name"] = name_entry.get().strip()
            self.logged_in_user["email"] = email_entry.get().strip()
            messagebox.showinfo("Success", "Profile updated successfully!")

        Button(self.form_frame, text="Update Profile",bg="#f5f6fa",font=("Times New Roman", 14,"bold"), command=update_profile).grid(row=4, column=0, columnspan=2, pady=10,padx=5,ipady=3)

    def register_window(self):
        self.clear_form_frame()
        self.form_frame.configure(bg="#f5f6fa")

        Label(self.form_frame,text="User Registration",font=("Times New Roman",18,"bold"),bg="#f5f6fa").grid(row=0,column=0,columnspan=2,padx=(20,2),pady=10,ipady=5,sticky="n")

        Label(self.form_frame, text="Full Name:", font=("Times New Roman", 12,"bold"), bg="#f5f6fa").grid(row=1, column=0,ipady=5,pady=5.5,padx=(15,0),sticky="e")
        name_entry = Entry(self.form_frame, width=30)
        name_entry.grid(row=1, column=1, pady=5.5,padx=(20,2),ipady=5,sticky="w")

        Label(self.form_frame, text="Mobile Number:", font=("Times New Roman", 12,"bold"), bg="#f5f6fa").grid(row=2, column=0,ipady=5,pady=5.5,padx=(15,0),sticky="e")
        mobile_entry = Entry(self.form_frame, width=30)
        mobile_entry.grid(row=2, column=1, pady=5.5,padx=(20,2),ipady=5,sticky="w")

        Label(self.form_frame, text="Email:", font=("Times New Roman", 12,"bold"), bg="#f5f6fa").grid(row=3, column=0,ipady=5,pady=5.5,padx=(15,0),sticky="e")
        email_entry = Entry(self.form_frame, width=30)
        email_entry.grid(row=3, column=1, pady=5,padx=(20,2),ipady=5,sticky="w")


        Label(self.form_frame, text="Password:", font=("Times New Roman", 12,"bold"), bg="#f5f6fa").grid(row=4, column=0,ipady=5,pady=5.5,padx=(15,0),sticky="e")
        password_entry = Entry(self.form_frame, width=30, show="*")
        password_entry.grid(row=4, column=1, pady=5,padx=(20,2),ipady=5,sticky="w")

        status_label = Label(self.form_frame, text="", fg="red", bg="#f5f6fa", font=("Times New Roman", 10))
        status_label.grid(row=6, column=0, columnspan=2, pady=5)

        def register_user():
            name = name_entry.get().strip()
            mobile = mobile_entry.get().strip()
            email = email_entry.get().strip()
            password = password_entry.get().strip()

            if not name or not mobile or not password:
               status_label.config(text="⚠ Please fill all required fields!")
               return

            conn = get_connection()
            if not conn:
               return
            cursor = conn.cursor()

            try:
                cursor.execute(
                "INSERT INTO users(name, contact, password, email) VALUES(%s, %s, %s, %s)",
                (name, mobile, password, email))

                conn.commit()
                status_label.config(text="✅ Registration successful!", fg="green")
            except Exception as e:
                status_label.config(text=f"⚠ Error: {e}", fg="red")
            finally:
                conn.close()

        Button(self.form_frame, text="Register", font=("Times New Roman", 12, "bold"),
           command=register_user).grid(row=5, column=0, columnspan=2, pady=10)


    def logout(self):
        self.logged_in_user = None
        messagebox.showinfo("Logout", "You have been logged out successfully!")
        self.clear_form_frame()


if __name__ == "__main__":
    app = CRUDApp()
    app.mainloop()


In [2]:
pip install tkcalendar pymysql Pillow

  Using cached tkcalendar-1.6.1-py3-none-any.whl.metadata (22 kB)
  Using cached babel-2.17.0-py3-none-any.whl.metadata (2.0 kB)
Using cached tkcalendar-1.6.1-py3-none-any.whl (40 kB)
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------------- -------------------------- 2.4/7.0 MB 11.0 MB/s eta 0:00:01
   ---------------------------- ----------- 5.0/7.0 MB 11.5 MB/s eta 0:00:01
   ---------------------------------------  6.8/7.0 MB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 10.4 MB/s  0:00:00
Using cached babel-2.17.0-py3-none-any.whl (10.2 MB)

   ---------------------------------------- 0/3 [Pillow]
   ---------------------------------------- 0/3 [Pillow]
   ---------------------------------------- 0/3 [Pillow]
   ---------------------------------------- 0/3 [Pillow]
   ---------------------------------------- 0/3 [Pillow]
   ---------------------------------------- 0/3 [Pillow]
   ---------------------------------------- 0/